In [1]:
import os
import time
import uuid
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

In [2]:
# Initialize Selenium WebDriver
def init_webdriver():
    options = Options()
    options.add_argument("--headless")  # Run in headless mode
    service = Service(EdgeChromiumDriverManager().install())
    driver = webdriver.Edge(service=service, options=options)
    return driver

In [3]:
# Load the page and extract HTML content
def get_page_content(driver, url):
    driver.get(url)
    time.sleep(10)  # Wait for the page to load
    html = driver.page_source
    return html

In [11]:
def extract_match_report_urls(html):
    base_url = "https://fbref.com"  # Root URL to prepend to relative links
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table", {"id": "matchlogs_for"})
    
    if not table:
        print("Table not found.")
        return []

    # Extract rows
    rows = table.find("tbody").findAll("tr")
    match_report_urls = []
    
    for row in rows:
        cells = row.findAll("td")
        if len(cells) > 0:
            match_report_cell = cells[-2]
            # Print cell HTML for debugging
            print(f"Cell HTML: {match_report_cell.prettify()}")
            
            link = match_report_cell.find("a")
            relative_url = link["href"] if link else None
            match_report_url = base_url + relative_url
            match_report_urls.append(match_report_url)
            print(f"Extracted URL: {match_report_url}")  # Debugging line
    
    return match_report_urls

In [13]:
# Update the main Excel file with Match Report URLs
def update_match_report_urls(df, urls):
    df["Match Report"] = urls
    df.to_excel("gwangju_fc_matches_2024_updated.xlsx", index=False)
    print("Updated main Excel file with Match Report URLs.")

In [55]:
def extract_gwangju_player_stats(html):
    soup = BeautifulSoup(html, 'html.parser')
    # Find the table with the specific caption
    table = None
    for tbl in soup.find_all('table'):
        caption = tbl.find('caption')
        if caption and caption.get_text(strip=True) == "Gwangju FC Player Stats Table":
            table = tbl
            break
    
    if not table:
        raise ValueError("Table with caption 'Gwangju FC Player Stats Table' not found.")
    
    # Skip the first tr (header with unnecessary grouping)
    header_rows = table.find('thead').find_all('tr')
    
    if len(header_rows) > 1:
        # The second row contains the actual column names
        actual_headers_row = header_rows[1]
        headers = [header.text.strip() for header in actual_headers_row.find_all('th')]
    else:
        # If there's no second row, fallback to the first row
        headers = [header.text.strip() for header in header_rows[0].find_all('th')]

    # Extract data rows
    rows = table.find('tbody').find_all('tr')
    data = []
    
    for row in rows:
        cells = [cell.text.strip() for cell in row.find_all(['td', 'th'])]  # Handle both headers and data
        
        # Replace empty values with 0
        cells = [cell if cell else '0' for cell in cells]  # <--- Replacement of empty data with '0'
        
        if cells:
            data.append(cells)

    # Adjust columns dynamically based on the data
    num_columns = len(data[0]) if data else 0
    if num_columns != len(headers):
        # Handle cases where data and headers mismatch
        print(f"Warning: Mismatch between headers ({len(headers)}) and data columns ({num_columns}). Adjusting headers.")
        headers = headers[:num_columns]  # Truncate headers to match data if necessary

    df = pd.DataFrame(data, columns=headers)
    return df

In [42]:
# Save the scraped match report table to a new Excel file
def save_report(df, opponent, match_number):
    folder_path = "Match-Reports"
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    filename = f"Report Gwangju FC - {opponent} - Matchday {match_number}.xlsx"
    file_path = os.path.join(folder_path, filename)
    df.to_excel(file_path, index=False)
    print(f"Saved match report to {file_path}")
    return file_path

In [17]:
# Scrape and save each match report as a new Excel file
def scrape_and_save_reports(df):
    driver = init_webdriver()
    try:
        for index, row in df.iterrows():
            match_report_url = row["Match Report"]
            opponent = row["Opponent"]
            match_number = index + 1  # Match number as a unique identifier
            
            # Skip if there's no valid match report URL
            if not pd.isna(match_report_url):
                
                # Check if the URL contains "stathead" and break if true
                if "stathead" in match_report_url:
                    print(f"Skipping Match {match_number} (not yet played): {match_report_url}")
                    break  # Stop processing further rows

                print(f"Processing Match {match_number}: Gwangju FC vs {opponent}")
                
                # Load match report page
                html = get_page_content(driver, match_report_url)
                match_report_df = extract_gwangju_player_stats(html)
                
                if match_report_df is not None:
                    # Save the scraped table to a new Excel file
                    file_path = save_report(match_report_df, opponent, match_number)
                    
                    # Optionally update the main Excel file with file paths or other metadata
                    df.at[index, "Match Report"] = file_path
                
            else:
                print(f"No match report found for Match {match_number}")
    
    finally:
        driver.quit()

In [58]:
# Main script
if __name__ == "__main__":
    # Load the main Excel file
    filename = "gwangju_fc_matches_2024.xlsx"
    df = pd.read_excel(filename)

    # Load the matchlogs_for page and extract URLs
    url = "https://fbref.com/en/squads/ae306ede/Gwangju-FC-Stats#all_matchlogs"
    driver = init_webdriver()
    html = get_page_content(driver, url)
    match_report_urls = extract_match_report_urls(html)
    driver.quit()

    # Update the main Excel file with URLs
    update_match_report_urls(df, match_report_urls)

    # Re-load the updated main Excel file with URLs
    df = pd.read_excel("gwangju_fc_matches_2024_updated.xlsx")

    # Scrape and save each match report
    scrape_and_save_reports(df)

Cell HTML: <td class="left group_start" data-stat="match_report">
 <a href="/en/matches/a93af5ad/Gwangju-FC-FC-Seoul-March-2-2024-K-League-1">
  Match Report
 </a>
</td>

Extracted URL: https://fbref.com/en/matches/a93af5ad/Gwangju-FC-FC-Seoul-March-2-2024-K-League-1
Cell HTML: <td class="left group_start" data-stat="match_report">
 <a href="/en/matches/f34b5326/Gwangju-FC-Gangwon-March-10-2024-K-League-1">
  Match Report
 </a>
</td>

Extracted URL: https://fbref.com/en/matches/f34b5326/Gwangju-FC-Gangwon-March-10-2024-K-League-1
Cell HTML: <td class="left group_start" data-stat="match_report">
 <a href="/en/matches/ec35b902/Pohang-Gwangju-FC-March-17-2024-K-League-1">
  Match Report
 </a>
</td>

Extracted URL: https://fbref.com/en/matches/ec35b902/Pohang-Gwangju-FC-March-17-2024-K-League-1
Cell HTML: <td class="left group_start" data-stat="match_report">
 <a href="/en/matches/630aeaa2/Gwangju-FC-Daegu-March-31-2024-K-League-1">
  Match Report
 </a>
</td>

Extracted URL: https://fbref.